# Performance dataframes

In [234]:
import os
import re
import json
import pandas as pd
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)
import numpy as np
from natsort import index_natsorted

## General parameters

In [238]:
nbr_sims = 5

instance_key = 'instance'
best_obj_key = 'best obj'
worst_obj_key = 'worst obj'
avg_obj_key = 'obj'
max_time_key = 'max time'
min_time_key = 'min time'
avg_time_key = 'time'
max_iter_key = 'max iter'
min_iter_key = 'min iter'
avg_iter_key = 'iter'
set_part_key = 'sp improv'

project_path = os.path.dirname(os.path.abspath('.'))
directory_path = '/output/solstorm/alns/performance/first/'

## Functions

In [247]:
def map_instance_to_data(run_path):
    instance_to_data = {}
    for file_name in os.listdir(run_path):
        split_name = re.split('_|\.', file_name)
        instance_name = split_name[0]
        is_history = split_name[2] == 'history'
        if is_history:
            with open(run_path + file_name) as file:
                history_json = json.load(file)
        
            obj = history_json['best_objective']
            time = history_json['runtime']
            it = history_json['number_of_iterations']
            set_part_improv = history_json['number_of_improvements_by_set_partitioning']

            if instance_name in instance_to_data:
                data = instance_to_data[instance_name]

                if obj < data[0]:
                    data[0] = obj
                if obj > data[1]:
                    data[1] = obj
                if time > data[3]:
                    data[3] = time
                if time < data[4]:
                    data[4] = time
                if it > data[6]:
                    data[6] = it
                if it < data[7]:
                    data[7] = it

                data[2] += obj
                data[5] += time
                data[8] += it
                data[9] += set_part_improv
                data[10] += 1
            else:
                instance_to_data[instance_name] = [obj, obj, obj, time, time, time, it, it, it, set_part_improv, 1]
    return instance_to_data

def generate_run_df(run_name):
    run_path = project_path + directory_path + run_name
    
    instance_to_data = map_instance_to_data(run_path)
    
    df = pd.DataFrame(columns=[instance_key, 
                               best_obj_key, worst_obj_key, avg_obj_key, 
                               max_time_key, min_time_key, avg_time_key, 
                               max_iter_key, min_iter_key, avg_iter_key])
    
    for instance in instance_to_data:
        data = instance_to_data[instance]
        best_objective = data[0]
        worst_objective = data[1]
        avg_objective = data[2] / nbr_sims
        max_time = data[3]
        min_time = data[4]
        avg_time = data[5] / nbr_sims
        max_iter = data[6]
        min_iter = data[7]
        avg_iter = data[8] / nbr_sims
        avg_set_part_improv = data[9] / nbr_sims
        
        if data[10] != nbr_sims:
            print(f'{instance} HAS LESS THAN FIVE SIMULATIONS!')
        
        row = pd.Series({instance_key: instance, 
                         best_obj_key: best_objective,
                         worst_obj_key: worst_objective,
                         avg_obj_key: avg_objective, 
                         max_time_key: max_time,
                         min_time_key: min_time,
                         avg_time_key: avg_time, 
                         max_iter_key: max_iter,
                         min_iter_key: min_iter,
                         avg_iter_key: avg_iter,
                         set_part_key: avg_set_part_improv})
        
        df = df.append(row, ignore_index=True)
    
    df = df.sort_values(by='instance',
                        key=lambda x: np.argsort(index_natsorted(df['instance'])),
                        inplace=False)
    df = df.reset_index(drop=True)
    
    mean_row = pd.Series({instance_key: 'Mean values',
                          best_obj_key: df[best_obj_key].mean(),
                          worst_obj_key: df[worst_obj_key].mean(),
                          avg_obj_key: df[avg_obj_key].mean(),
                          max_time_key: df[max_time_key].mean(),
                          min_time_key: df[min_time_key].mean(),
                          avg_time_key: df[avg_time_key].mean(),
                          max_iter_key: df[max_iter_key].mean(),
                          min_iter_key: df[min_iter_key].mean(),
                          avg_iter_key: df[avg_iter_key].mean(),
                          set_part_key: df[set_part_key].mean()})
    df = df.append(mean_row, ignore_index=True)
    df = df.round(1)
    return df

def merge_dfs(dfs):
    df_copies = [df.copy() for df in dfs]
    for df in df_copies:
        df.drop([best_obj_key, worst_obj_key, max_time_key, min_time_key, max_iter_key, min_iter_key], 
                axis=1, inplace=True)
    df_total = pd.concat(df_copies, axis=1)
    
    # Drop duplicate instance columns
    li = [i for i in range(4, len(df_total.columns), 4)]
    df_total = df_total.iloc[:, [j for j, c in enumerate(df_total.columns) if j not in li]]
    
    df_total = df_total.round(1)
    return df_total

def aggregate_df_by_instance_group(df):
    instance_size_to_data = {}
    for idx, row in df.iterrows():
        instance_name = row[instance_key]

        if instance_name == 'Mean values':
            continue
        
        split_name = re.split('-', instance_name)
        instance_size = split_name[0]
        
        best_obj = row[best_obj_key]
        worst_obj = row[worst_obj_key]
        avg_obj = row[avg_obj_key]
        max_time = row[max_time_key]
        min_time = row[min_time_key]
        avg_time = row[avg_time_key]
        max_iter = row[max_iter_key]
        min_iter = row[min_iter_key]
        avg_iter = row[avg_iter_key]
        avg_set_part_improv = row[set_part_key]
        
        if instance_size in instance_size_to_data:
            data = instance_size_to_data[instance_size]
            data[0] += best_obj
            data[1] += worst_obj
            data[2] += avg_obj
            data[3] += max_time
            data[4] += min_time
            data[5] += avg_time
            data[6] += max_iter
            data[7] += min_iter
            data[8] += avg_iter
            data[9] += avg_set_part_improv
            data[10] += 1  # Number of times encountered instance size
        else:
            data = [best_obj, worst_obj, avg_obj, max_time, min_time, avg_time, max_iter, min_iter, avg_iter, avg_set_part_improv, 1]
            instance_size_to_data[instance_size] = data

    df = pd.DataFrame(columns=['instance_group', best_obj_key, worst_obj_key, avg_obj_key, max_time_key, min_time_key,
                               avg_time_key, max_iter_key, min_iter_key, avg_iter_key, set_part_key])
    
    for instance_size in instance_size_to_data:
        data = instance_size_to_data[instance_size]
        nbr_sims = data[10]
        if nbr_sims < 5:
            print(f'{instance_size} INSTANCE_SIZE LESS THAN FIVE INSTANCES!')
        
        row = pd.Series({'instance_group': instance_size, 
                         best_obj_key: data[0] / nbr_sims,
                         worst_obj_key: data[1] / nbr_sims,
                         avg_obj_key: data[2] / nbr_sims, 
                         max_time_key: data[3] / nbr_sims,
                         min_time_key: data[4] / nbr_sims,
                         avg_time_key: data[5] / nbr_sims, 
                         max_iter_key: data[6] / nbr_sims,
                         min_iter_key: data[7] / nbr_sims,
                         avg_iter_key: data[8] / nbr_sims,
                         set_part_key: data[9] / nbr_sims})
    
        df = df.append(row, ignore_index=True)
        
    mean_row = pd.Series({'instance_group': 'Mean values',
                          best_obj_key: df[best_obj_key].mean(),
                          worst_obj_key: df[worst_obj_key].mean(),
                          avg_obj_key: df[avg_obj_key].mean(),
                          max_time_key: df[max_time_key].mean(),
                          min_time_key: df[min_time_key].mean(),
                          avg_time_key: df[avg_time_key].mean(),
                          max_iter_key: df[max_iter_key].mean(),
                          min_iter_key: df[min_iter_key].mean(),
                          avg_iter_key: df[avg_iter_key].mean(),
                          set_part_key: df[set_part_key].mean()})
    df = df.append(mean_row, ignore_index=True)
    df = df.round(1)
    return df

## ALNS baseline

In [248]:
run_baseline_name = 'baseline/'
run_baseline_df = generate_run_df(run_baseline_name)
run_baseline_agg_df = aggregate_df_by_instance_group(run_baseline_df)

In [249]:
run_baseline_df

,instance,best obj,worst obj,obj,max time,min time,time,max iter,min iter,iter,sp improv
0,5-5-1-1,2519.9,2519.9,2519.9,0.7,0.7,0.7,5000,5000,5000.0,0.0
1,5-5-1-2,2214.2,2214.2,2214.2,0.7,0.7,0.7,5000,5000,5000.0,0.0
2,5-6-1-1,1939.6,1939.6,1939.6,0.9,0.8,0.9,5000,5000,5000.0,0.0
3,5-7-1-1,2709.7,2709.7,2709.7,1.4,1.3,1.4,5000,5000,5000.0,0.0
4,5-7-1-2,1704.6,1704.6,1704.6,1.2,1.2,1.2,5000,5000,5000.0,0.0
5,7-8-1-1,2046.9,2046.9,2046.9,2.1,1.8,1.9,5000,5000,5000.0,0.0
6,7-8-1-2,1971.3,1971.3,1971.3,4.9,4.4,4.6,5000,5000,5000.0,0.0
7,7-8-1-3,2071.9,2071.9,2071.9,3.5,3.3,3.4,5000,5000,5000.0,0.0
8,7-9-1-1,2170.4,2170.4,2170.4,4.6,3.7,4.1,5000,5000,5000.0,0.0
9,7-9-1-2,2206.1,2206.1,2206.1,5.9,5.3,5.6,5000,5000,5000.0,0.0


In [250]:
run_baseline_agg_df

,instance_group,best obj,worst obj,obj,max time,min time,time,max iter,min iter,iter,sp improv
0,5,2217.6,2217.6,2217.6,1.0,0.9,1.0,5000.0,5000.0,5000.0,0.0
1,7,2093.3,2093.3,2093.3,4.2,3.7,3.9,5000.0,5000.0,5000.0,0.0
2,9,5611.0,5627.5,5617.6,8.3,7.5,7.8,5000.0,5000.0,5000.0,0.0
3,11,3398.8,3477.0,3449.7,26.1,22.0,24.0,5000.0,5000.0,5000.0,0.0
4,13,3748.3,3947.4,3865.5,55.0,46.3,50.0,5000.0,5000.0,5000.0,0.0
5,15,6037.5,8355.7,7436.0,65.2,55.2,60.6,5000.0,5000.0,5000.0,0.0
6,17,4861.8,4966.7,4920.4,114.1,96.0,105.2,5000.0,5000.0,5000.0,0.0
7,19,4909.2,5137.4,5050.4,141.1,122.6,130.3,5000.0,5000.0,5000.0,0.0
8,21,7415.5,9308.6,8628.6,168.2,152.4,161.2,5000.0,5000.0,5000.0,0.0
9,23,5791.3,5974.3,5886.4,248.4,224.9,235.5,5000.0,5000.0,5000.0,0.0


## Sequential ALNS

In [216]:
run_sequential_name = 'sequential/'
run_sequential_df = generate_run_df(run_sequential_name)
run_sequential_agg_df = aggregate_df_by_instance_group(run_sequential_df)

In [217]:
run_sequential_df

,instance,best obj,worst obj,obj,max time,min time,time,max iter,min iter,iter
0,5-5-1-1,2519.9,2519.9,2519.9,0.7,0.7,0.7,5000,5000,5000.0
1,5-5-1-2,2214.2,2214.2,2214.2,0.7,0.7,0.7,5000,5000,5000.0
2,5-6-1-1,1939.6,1939.6,1939.6,0.8,0.8,0.8,5000,5000,5000.0
3,5-7-1-1,2709.7,2709.7,2709.7,1.4,1.3,1.3,5000,5000,5000.0
4,5-7-1-2,1704.6,1704.6,1704.6,1.2,1.2,1.2,5000,5000,5000.0
5,7-8-1-1,2046.9,2046.9,2046.9,2.2,1.9,2.1,5000,5000,5000.0
6,7-8-1-2,1971.3,1971.3,1971.3,5.6,5.4,5.5,5000,5000,5000.0
7,7-8-1-3,2071.9,2071.9,2071.9,3.6,3.4,3.5,5000,5000,5000.0
8,7-9-1-1,2170.4,2170.4,2170.4,4.8,4.7,4.8,5000,5000,5000.0
9,7-9-1-2,2206.1,2206.1,2206.1,6.7,6.3,6.6,5000,5000,5000.0


In [218]:
run_sequential_agg_df

,instance_group,best obj,worst obj,obj,max time,min time,time,max iter,min iter,iter
0,5,2217.6,2217.6,2217.6,1.0,0.9,0.9,5000.0,5000.0,5000.0
1,7,2093.3,2093.3,2093.3,4.6,4.3,4.5,5000.0,5000.0,5000.0
2,9,5627.5,5627.5,5627.5,10.8,9.5,10.1,5000.0,5000.0,5000.0
3,11,3517.5,3517.5,3517.5,29.9,26.0,28.0,5000.0,5000.0,5000.0
4,13,3970.5,3970.5,3970.5,67.4,56.4,61.7,5000.0,5000.0,5000.0
5,15,6179.0,8399.9,7944.4,86.7,74.9,80.7,5000.0,5000.0,5000.0
6,17,4827.9,4986.8,4943.2,151.6,124.0,138.8,5000.0,5000.0,5000.0
7,19,4926.9,5173.1,5095.8,190.6,166.2,178.9,5000.0,5000.0,5000.0
8,21,8362.7,9332.5,9116.9,238.3,201.3,218.0,5000.0,5000.0,5000.0
9,23,5619.4,5995.6,5871.9,361.2,326.2,342.6,5000.0,5000.0,5000.0


## LNS

In [220]:
run_lns_name = 'lns/'
run_lns_df = generate_run_df(run_lns_name)
run_lns_agg_df = aggregate_df_by_instance_group(run_lns_df)

5-7-1-2 HAS LESS THAN FIVE SIMULATIONS!


In [221]:
run_lns_df

,instance,best obj,worst obj,obj,max time,min time,time,max iter,min iter,iter
0,5-5-1-1,2519.9,2519.9,2519.9,0.8,0.7,0.7,5000,5000,5000.0
1,5-5-1-2,2214.2,2214.2,2214.2,0.7,0.7,0.7,5000,5000,5000.0
2,5-6-1-1,1939.6,1939.6,1939.6,0.9,0.8,0.8,5000,5000,5000.0
3,5-7-1-1,2709.7,2709.7,2709.7,1.5,1.4,1.4,5000,5000,5000.0
4,5-7-1-2,1704.6,1704.6,1363.7,1.2,1.1,0.9,5000,5000,4000.0
5,7-8-1-1,2046.9,2046.9,2046.9,2.0,1.8,1.9,5000,5000,5000.0
6,7-8-1-2,1971.3,1971.3,1971.3,5.0,4.4,4.6,5000,5000,5000.0
7,7-8-1-3,2071.9,2071.9,2071.9,3.6,3.3,3.4,5000,5000,5000.0
8,7-9-1-1,2170.4,2170.4,2170.4,4.4,3.8,4.0,5000,5000,5000.0
9,7-9-1-2,2206.1,2206.1,2206.1,5.8,5.2,5.6,5000,5000,5000.0


In [222]:
run_lns_agg_df

,instance_group,best obj,worst obj,obj,max time,min time,time,max iter,min iter,iter
0,5,2217.6,2217.6,2149.4,1.0,0.9,0.9,5000.0,5000.0,4800.0
1,7,2093.3,2093.3,2093.3,4.2,3.7,3.9,5000.0,5000.0,5000.0
2,9,5611.0,5627.5,5620.9,8.4,7.1,7.8,5000.0,5000.0,5000.0
3,11,3399.9,3475.9,3446.8,25.4,20.2,23.4,5000.0,5000.0,5000.0
4,13,3765.6,3917.1,3862.6,51.6,42.2,47.6,5000.0,5000.0,5000.0
5,15,6033.8,8385.5,7866.3,66.6,56.0,61.7,5000.0,5000.0,5000.0
6,17,4802.0,4965.3,4903.5,112.4,97.3,105.4,5000.0,5000.0,5000.0
7,19,4898.5,5128.2,5051.5,130.7,117.0,125.2,5000.0,5000.0,5000.0
8,21,7749.0,9283.7,8678.3,168.0,148.6,158.5,5000.0,5000.0,5000.0
9,23,5702.5,5973.8,5857.2,241.4,222.7,231.2,5000.0,5000.0,5000.0


## ALNS + local search

In [198]:
run_ls_name = 'ls/'
run_ls_df = generate_run_df(run_ls_name)
run_ls_agg_df = aggregate_df_by_instance_group(run_ls_df)

17-22-3-1 HAS LESS THAN FIVE SIMULATIONS!


In [199]:
run_ls_df

,instance,best obj,worst obj,obj,max time,min time,time,max iter,min iter,iter
0,5-5-1-1,2519.9,2519.9,2519.9,1.5,1.4,1.5,5000,5000,5000.0
1,5-5-1-2,2214.2,2214.2,2214.2,1.3,1.3,1.3,5000,5000,5000.0
2,5-6-1-1,1939.6,1939.6,1939.6,1.6,1.5,1.5,5000,5000,5000.0
3,5-7-1-1,2709.7,2709.7,2709.7,2.4,2.2,2.3,5000,5000,5000.0
4,5-7-1-2,1704.6,1704.6,1704.6,1.9,1.8,1.8,5000,5000,5000.0
5,7-8-1-1,2046.9,2046.9,2046.9,4.0,3.7,3.8,5000,5000,5000.0
6,7-8-1-2,1971.3,1971.3,1971.3,6.9,6.2,6.7,5000,5000,5000.0
7,7-8-1-3,2071.9,2071.9,2071.9,5.8,5.5,5.7,5000,5000,5000.0
8,7-9-1-1,2170.4,2170.4,2170.4,6.4,6.0,6.3,5000,5000,5000.0
9,7-9-1-2,2206.1,2206.1,2206.1,8.5,7.9,8.1,5000,5000,5000.0


In [200]:
run_ls_agg_df

,instance_group,best obj,worst obj,obj,max time,min time,time,max iter,min iter,iter
0,5,2217.60,2217.60,2217.60,1.74,1.64,1.68,5000.0,5000.0,5000.00
1,7,2093.32,2093.32,2093.32,6.32,5.86,6.12,5000.0,5000.0,5000.00
2,9,5467.98,5627.54,5567.64,11.36,10.32,10.86,5000.0,5000.0,5000.00
3,11,3425.80,3504.84,3475.66,45.26,36.22,40.94,5000.0,5000.0,5000.00
4,13,3668.34,3938.18,3856.16,87.72,73.36,80.10,5000.0,5000.0,5000.00
5,15,8139.28,8359.94,8278.50,118.78,104.04,110.52,5000.0,5000.0,5000.00
6,17,4516.38,4933.96,4570.42,226.72,173.44,190.18,5000.0,5000.0,4800.00
7,19,4673.82,5112.54,4962.62,284.98,222.56,260.80,5000.0,5000.0,5000.00
8,21,7766.62,9277.08,8755.56,280.04,227.12,255.26,5000.0,5000.0,5000.00
9,23,5500.32,5891.62,5710.60,467.76,385.24,429.88,5000.0,4992.8,4998.56


## ALNS + set partitioning

In [251]:
run_sp_name = 'sp/'
run_sp_df = generate_run_df(run_sp_name)
run_sp_agg_df = aggregate_df_by_instance_group(run_sp_df)

In [252]:
run_sp_df

,instance,best obj,worst obj,obj,max time,min time,time,max iter,min iter,iter,sp improv
0,5-5-1-1,2519.9,2519.9,2519.9,0.9,0.8,0.8,5000,5000,5000.0,0.0
1,5-5-1-2,2214.2,2214.2,2214.2,0.8,0.7,0.8,5000,5000,5000.0,0.0
2,5-6-1-1,1939.6,1939.6,1939.6,0.9,0.8,0.9,5000,5000,5000.0,0.0
3,5-7-1-1,2709.7,2709.7,2709.7,1.5,1.4,1.4,5000,5000,5000.0,0.0
4,5-7-1-2,1704.6,1704.6,1704.6,1.3,1.2,1.2,5000,5000,5000.0,0.0
5,7-8-1-1,2046.9,2046.9,2046.9,2.1,1.8,2.0,5000,5000,5000.0,0.0
6,7-8-1-2,1971.3,1971.3,1971.3,4.8,4.3,4.6,5000,5000,5000.0,0.0
7,7-8-1-3,2071.9,2071.9,2071.9,3.9,3.3,3.5,5000,5000,5000.0,0.0
8,7-9-1-1,2170.4,2170.4,2170.4,4.3,4.0,4.1,5000,5000,5000.0,0.0
9,7-9-1-2,2206.1,2206.1,2206.1,5.8,5.1,5.5,5000,5000,5000.0,0.0


In [253]:
run_sp_agg_df

,instance_group,best obj,worst obj,obj,max time,min time,time,max iter,min iter,iter,sp improv
0,5,2217.6,2217.6,2217.6,1.1,1.0,1.0,5000.0,5000.0,5000.0,0.0
1,7,2093.3,2093.3,2093.3,4.2,3.7,3.9,5000.0,5000.0,5000.0,0.0
2,9,5611.0,5627.5,5614.3,8.3,7.2,7.8,5000.0,5000.0,5000.0,0.0
3,11,3390.5,3467.3,3436.3,25.7,21.7,23.8,5000.0,5000.0,5000.0,0.2
4,13,3744.9,3925.3,3853.4,54.8,49.6,52.1,5000.0,5000.0,5000.0,0.2
5,15,6042.0,8356.4,7506.7,66.7,58.1,62.9,5000.0,5000.0,5000.0,1.1
6,17,4831.4,4960.9,4917.4,115.8,95.1,103.7,5000.0,5000.0,5000.0,1.4
7,19,4871.5,5128.4,5054.6,136.9,124.1,131.4,5000.0,5000.0,5000.0,1.9
8,21,7931.6,9242.3,8802.5,176.7,153.7,163.3,5000.0,5000.0,5000.0,2.7
9,23,5539.5,5938.0,5796.4,251.0,231.7,240.6,5000.0,5000.0,5000.0,2.6


## ALNS + local search + set partitioning

In [226]:
run_ls_sp_name = 'ls/'
run_ls_sp_df = generate_run_df(run_ls_sp_name)
run_ls_sp_agg_df = aggregate_df_by_instance_group(run_ls_sp_df)

17-22-3-1 HAS LESS THAN FIVE SIMULATIONS!


In [227]:
run_ls_sp_df

,instance,best obj,worst obj,obj,max time,min time,time,max iter,min iter,iter
0,5-5-1-1,2519.9,2519.9,2519.9,1.5,1.4,1.5,5000,5000,5000.0
1,5-5-1-2,2214.2,2214.2,2214.2,1.3,1.3,1.3,5000,5000,5000.0
2,5-6-1-1,1939.6,1939.6,1939.6,1.6,1.5,1.5,5000,5000,5000.0
3,5-7-1-1,2709.7,2709.7,2709.7,2.4,2.2,2.3,5000,5000,5000.0
4,5-7-1-2,1704.6,1704.6,1704.6,1.9,1.8,1.8,5000,5000,5000.0
5,7-8-1-1,2046.9,2046.9,2046.9,4.0,3.7,3.8,5000,5000,5000.0
6,7-8-1-2,1971.3,1971.3,1971.3,6.9,6.2,6.7,5000,5000,5000.0
7,7-8-1-3,2071.9,2071.9,2071.9,5.8,5.5,5.7,5000,5000,5000.0
8,7-9-1-1,2170.4,2170.4,2170.4,6.4,6.0,6.3,5000,5000,5000.0
9,7-9-1-2,2206.1,2206.1,2206.1,8.5,7.9,8.1,5000,5000,5000.0


In [228]:
run_ls_sp_agg_df

,instance_group,best obj,worst obj,obj,max time,min time,time,max iter,min iter,iter
0,5,2217.6,2217.6,2217.6,1.7,1.6,1.7,5000.0,5000.0,5000.0
1,7,2093.3,2093.3,2093.3,6.3,5.9,6.1,5000.0,5000.0,5000.0
2,9,5468.0,5627.5,5567.6,11.4,10.3,10.9,5000.0,5000.0,5000.0
3,11,3425.8,3504.8,3475.7,45.3,36.2,40.9,5000.0,5000.0,5000.0
4,13,3668.3,3938.2,3856.2,87.7,73.4,80.1,5000.0,5000.0,5000.0
5,15,8139.3,8359.9,8278.5,118.8,104.0,110.5,5000.0,5000.0,5000.0
6,17,4516.4,4934.0,4570.4,226.7,173.4,190.2,5000.0,5000.0,4800.0
7,19,4673.8,5112.5,4962.6,285.0,222.6,260.8,5000.0,5000.0,5000.0
8,21,7766.6,9277.1,8755.6,280.0,227.1,255.3,5000.0,5000.0,5000.0
9,23,5500.3,5891.6,5710.6,467.8,385.2,429.9,5000.0,4992.8,4998.6


## Parallel vs. sequential heuristics

In [229]:
baseline_sequential_df = merge_dfs([run_baseline_df, run_sequential_df])
baseline_sequential_agg_df = merge_dfs([run_baseline_agg_df, run_sequential_agg_df])

In [230]:
baseline_sequential_df

,instance,obj,time,iter,obj,time,iter
0,5-5-1-1,2519.9,0.7,5000.0,2519.9,0.7,5000.0
1,5-5-1-2,2214.2,0.7,5000.0,2214.2,0.7,5000.0
2,5-6-1-1,1939.6,0.9,5000.0,1939.6,0.8,5000.0
3,5-7-1-1,2709.7,1.4,5000.0,2709.7,1.3,5000.0
4,5-7-1-2,1704.6,1.2,5000.0,1704.6,1.2,5000.0
5,7-8-1-1,2046.9,1.9,5000.0,2046.9,2.1,5000.0
6,7-8-1-2,1971.3,4.6,5000.0,1971.3,5.5,5000.0
7,7-8-1-3,2071.9,3.4,5000.0,2071.9,3.5,5000.0
8,7-9-1-1,2170.4,4.1,5000.0,2170.4,4.8,5000.0
9,7-9-1-2,2206.1,5.6,5000.0,2206.1,6.6,5000.0


In [231]:
baseline_sequential_agg_df

,instance_group,obj,time,iter,obj,time,iter
0,5,2217.6,1.0,5000.0,2217.6,0.9,5000.0
1,7,2093.3,3.9,5000.0,2093.3,4.5,5000.0
2,9,5617.6,7.8,5000.0,5627.5,10.1,5000.0
3,11,3449.7,24.0,5000.0,3517.5,28.0,5000.0
4,13,3865.5,50.0,5000.0,3970.5,61.7,5000.0
5,15,7436.0,60.6,5000.0,7944.4,80.7,5000.0
6,17,4920.4,105.2,5000.0,4943.2,138.8,5000.0
7,19,5050.4,130.3,5000.0,5095.8,178.9,5000.0
8,21,8628.6,161.2,5000.0,9116.9,218.0,5000.0
9,23,5886.4,235.5,5000.0,5871.9,342.6,5000.0


## ALNS vs. LNS

In [232]:
baseline_lns_df = merge_dfs([run_baseline_df, run_lns_df])
baseline_lns_df

,instance,obj,time,iter,obj,time,iter
0,5-5-1-1,2519.9,0.7,5000.0,2519.9,0.7,5000.0
1,5-5-1-2,2214.2,0.7,5000.0,2214.2,0.7,5000.0
2,5-6-1-1,1939.6,0.9,5000.0,1939.6,0.8,5000.0
3,5-7-1-1,2709.7,1.4,5000.0,2709.7,1.4,5000.0
4,5-7-1-2,1704.6,1.2,5000.0,1363.7,0.9,4000.0
5,7-8-1-1,2046.9,1.9,5000.0,2046.9,1.9,5000.0
6,7-8-1-2,1971.3,4.6,5000.0,1971.3,4.6,5000.0
7,7-8-1-3,2071.9,3.4,5000.0,2071.9,3.4,5000.0
8,7-9-1-1,2170.4,4.1,5000.0,2170.4,4.0,5000.0
9,7-9-1-2,2206.1,5.6,5000.0,2206.1,5.6,5000.0


## ALNS vs. ALNS + local search

In [174]:
baseline_ls_df = merge_dfs([run_baseline_df, run_ls_df])
baseline_ls_df

17-22-3-1 HAS LESS THAN FIVE SIMULATIONS!


## ALNS vs. ALNS + set partitioning

## ALNS vs. ALNS + local search + set partitioning